# Data Collaction Code

In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import csv
import time

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def calculate_angle(x, y):
    return math.atan2(y, x) * 180 / math.pi  # Convert radians to degrees

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=2)

offset = 20
imgSize = 300

# Create a CSV file for saving landmarks, distances, and angles
csv_file = None
csv_writer = None

# Lists to store distances and angles for each landmark
distances_list = [[] for _ in range(21)]
angles_list = [[] for _ in range(21)]

# Timer variables
start_time = None
data_collection_duration = 30  # 30 seconds
extraction_running = False

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if extraction_running:
        # Reset imgWhiteLeft and imgWhiteRight for each frame
        imgWhiteLeft = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgWhiteRight = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        for hand in hands:
            handType = hand['type']
            landmarks = hand['lmList']

            # Extract bounding box information
            x, y, w, h = hand['bbox']

            # Calculate scaling factors for X and Y coordinates
            scaleX = imgSize / w
            scaleY = imgSize / h

            # Scale the landmarks to fit the resized frame
            scaled_landmarks = [[int(scaleX * point[0]), int(scaleY * point[1])] for point in landmarks]

            # Flatten the list of scaled landmarks into [X1, Y1, Distance1, Angle1, X2, Y2, Distance2, Angle2, ..., X20, Y20, Distance20, Angle20]
            flattened_scaled_landmarks = []
            for point in scaled_landmarks:
                x, y = point
                distance = calculate_distance(0, 0, x, y)
                angle = calculate_angle(x, y)
                flattened_scaled_landmarks.extend([x, y, distance, angle])

                # Store distances and angles in lists
                distances_list[len(flattened_scaled_landmarks) // 4 - 1].append(distance)
                angles_list[len(flattened_scaled_landmarks) // 4 - 1].append(angle)

            x, y, w, h = hand['bbox']
            imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

            if not imgCrop.size == 0:
                aspectRatio = h / w

                if aspectRatio > 1:
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhiteLeft[:, wGap:wCal + wGap, :] = imgResize if handType == 'Left' else imgWhiteLeft[:, wGap:wCal + wGap, :]
                    imgWhiteRight[:, wGap:wCal + wGap, :] = imgResize if handType == 'Right' else imgWhiteRight[:, wGap:wCal + wGap, :]
                else:
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhiteLeft[hGap:hCal + hGap, :, :] = imgResize if handType == 'Left' else imgWhiteLeft[hGap:hCal + hGap, :, :]
                    imgWhiteRight[hGap:hCal + hGap, :, :] = imgResize if handType == 'Right' else imgWhiteRight[hGap:hCal + hGap, :, :]
            else:
                print("Warning: Empty image. Skipping resizing.")

            # Write data to CSV file
            row_data = [handType] + flattened_scaled_landmarks
            csv_writer.writerow(row_data)

        if imgWhiteLeft.shape[0] > 0 and imgWhiteLeft.shape[1] > 0:
            cv2.imshow("ImageWhiteLeft", imgWhiteLeft)
            cv2.imshow("ImageWhiteRight", imgWhiteRight)
        else:
            print("Warning: Invalid image dimensions. Skipping display.")

        # Display the timer on the window frame
        remaining_time = max(0, data_collection_duration - (time.time() - start_time))
        cv2.putText(img, f"Time Remaining: {int(remaining_time)}s", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Image", img)

    key = cv2.waitKey(1)

    if key == ord('s') and not extraction_running:  # Press 's' to start data collection
        start_time = time.time()
        extraction_running = True

        # Create a new CSV file for each extraction session
        csv_file_path = f'hand_landmarks_Key_Points_H_{int(start_time)}.csv'
        csv_file = open(csv_file_path, 'w', newline='')
        csv_writer = csv.writer(csv_file)

        # Write header row
        header_row = ['HandType']
        for i in range(21):
            header_row.extend([f'Landmarks_{i}_X', f'Landmarks_{i}_Y', f'Landmarks_{i}_Distance', f'Landmarks_{i}_Angle'])
        csv_writer.writerow(header_row)

    # Check if the specified data collection duration has passed
    if extraction_running and start_time is not None and (time.time() - start_time) >= data_collection_duration:
        # Reset the timer and stop extraction
        start_time = None
        extraction_running = False

        # Close the CSV file
        if csv_file is not None:
            csv_file.close()

    if key == ord('q'):  # Press 'q' to exit the loop
        # Close the CSV file before exiting
        if csv_file is not None:
            csv_file.close()

        # Release resources
        cap.release()
        cv2.destroyAllWindows()
        break


#  Model Build Code

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib  # Import the joblib library for saving and loading models

In [ ]:
# Load your dataset
df = pd.read_csv('Alphabet.csv')

In [ ]:
df

,Landmarks_0_X,Landmarks_0_Y,Landmarks_0_Distance,Landmarks_0_Angle,Landmarks_1_X,Landmarks_1_Y,Landmarks_1_Distance,Landmarks_1_Angle,Landmarks_2_X,Landmarks_2_Y,...,Landmarks_18_Angle,Landmarks_19_X,Landmarks_19_Y,Landmarks_19_Distance,Landmarks_19_Angle,Landmarks_20_X,Landmarks_20_Y,Landmarks_20_Distance,Landmarks_20_Angle,Output
0,959,1051,1422.772645,47.620664,846,985,1298.437908,49.341271,763,893,...,47.337941,871,969,1302.920565,48.048742,871,966,1300.690970,47.960400,1
1,575,1196,1327.042200,64.323184,693,1150,1342.664887,58.926468,776,1057,...,61.400014,619,1160,1314.823562,61.914696,594,1128,1274.841167,62.228900,1
2,952,1022,1396.706125,47.030914,839,961,1275.712350,48.877450,757,869,...,46.837398,861,942,1276.199436,47.572290,859,942,1274.850972,47.638638,1
3,605,1279,1414.873139,64.684643,717,1211,1407.341465,59.371384,794,1124,...,61.813629,651,1234,1395.190668,62.185992,625,1203,1355.667363,62.546567,1
4,963,1042,1418.849182,47.256372,851,980,1297.921800,49.030018,769,887,...,48.304258,869,983,1312.040396,48.522402,889,971,1316.496107,47.524308,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9133,237,1193,1216.313282,78.763974,338,1165,1213.041219,73.821036,402,1094,...,66.731314,467,986,1091.001833,64.656322,505,989,1110.471071,62.950381,26
9134,3080,520,3123.587681,9.582945,2790,472,2829.643794,9.602133,2780,420,...,6.385998,3020,305,3035.362417,5.766940,3030,277,3042.635207,5.223412,26
9135,234,1184,1206.901819,78.820412,329,1157,1202.867407,74.126575,395,1083,...,66.865205,464,979,1083.391434,64.641247,504,982,1103.784399,62.831369,26
9136,3186,520,3228.156750,9.269742,2896,474,2934.534375,9.295411,2906,423,...,6.160341,3124,304,3138.756442,5.558018,3144,277,3156.178861,5.035005,26


In [ ]:
# Separate features (X) and target variable (y)
X = df.iloc[:, :-1]
y = df['Output']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)


In [ ]:
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy*100}")

Model Accuracy: 98.7964989059081


In [ ]:
# Save the trained model to a file
joblib.dump(model, 'ISL_Alphabet26_model.joblib')

['ISL_Alphabet26_model.joblib']

# Model Evalution Code

In [ ]:
import cv2
import numpy as np
import joblib
import math
from cvzone.HandTrackingModule import HandDetector

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def calculate_angle(x, y):
    return math.atan2(y, x) * 180 / math.pi

# Load the saved model
loaded_model = joblib.load('ISL_Alphabet_model.joblib')

# Dictionary to map predicted class to corresponding text
class_mapping = {
    1: "This symbol is A",
    2: "This symbol is B",
    3: "This symbol is C",
    4: "This symbol is D",
    5: "This symbol is E",
    6: "This symbol is F",
    7: "This symbol is G",
    8: "This symbol is H",
    9: "This symbol is I",
    10: "This symbol is J",
    11: "This symbol is K",
    12: "This symbol is L",
    13: "This symbol is M",
    14: "This symbol is N",
    15: "This symbol is O",
    16: "This symbol is P",
    17: "This symbol is Q",
    18: "This symbol is R",
    19: "This symbol is S",
    20: "This symbol is T",
    21: "This symbol is U",
    22: "This symbol is V",
    23: "This symbol is W",
    24: "This symbol is X",
    25: "This symbol is Y",
    26: "This symbol is Z"
}

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=2)

offset = 20
imgSize = 300

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    for hand in hands:
        handType = hand['type']
        landmarks = hand['lmList']

        x, y, w, h = hand['bbox']
        scaleX = imgSize / w
        scaleY = imgSize / h

        scaled_landmarks = [[int(scaleX * point[0]), int(scaleY * point[1])] for point in landmarks]

        flattened_scaled_landmarks = []
        for point in scaled_landmarks:
            x, y = point
            distance = calculate_distance(0, 0, x, y)
            angle = calculate_angle(x, y)
            flattened_scaled_landmarks.extend([x, y, distance, angle])

        predictions = loaded_model.predict([flattened_scaled_landmarks])
        predicted_class = int(predictions[0])

        text_to_display = class_mapping.get(predicted_class, "Unknown")
        cv2.putText(img, f"Predicted: {text_to_display}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Image", img)

    key = cv2.waitKey(1)

    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break


C:\Users\mukhe\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\mukhe\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\mukhe\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\mukhe\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\mukhe\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted w

# Word Application

In [ ]:
import cv2
import numpy as np
import joblib
import math
import tkinter as tk
from PIL import Image, ImageTk
from cvzone.HandTrackingModule import HandDetector

# Define the image size for scaling landmarks
imgSize = 300

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def calculate_angle(x, y):
    return math.atan2(y, x) * 180 / math.pi

def calculate_landmarks(hand):
    landmarks = hand['lmList']
    x, y, w, h = hand['bbox']
    scaleX = imgSize / w
    scaleY = imgSize / h
    scaled_landmarks = [[int(scaleX * point[0]), int(scaleY * point[1])] for point in landmarks]
    flattened_scaled_landmarks = []
    for point in scaled_landmarks:
        x, y = point
        distance = calculate_distance(0, 0, x, y)
        angle = calculate_angle(x, y)
        flattened_scaled_landmarks.extend([x, y, distance, angle])
    return flattened_scaled_landmarks

def predict_numbers(hand):
    flattened_scaled_landmarks = calculate_landmarks(hand)
    predictions = number_model.predict([flattened_scaled_landmarks])
    predicted_number = number_class_mapping.get(int(predictions[0]), "Unknown")
    return predicted_number

def predict_alphabets(hands):
    if len(hands) == 2:
        flattened_scaled_landmarks = calculate_landmarks(hands[0])  # Modify this if you want to use both hands for prediction
        predictions = alphabet_model.predict([flattened_scaled_landmarks])
        predicted_alphabet = alphabet_class_mapping.get(int(predictions[0]), "Unknown")
    else:
        predicted_alphabet = "Please show both hands for alphabet sign prediction"
    return predicted_alphabet

def predict_word(hand):
    flattened_scaled_landmarks = calculate_landmarks(hand)
    predictions = word_model.predict([flattened_scaled_landmarks])
    predicted_word = word_class_mapping.get(int(predictions[0]), "Unknown")
    return predicted_word

def start_detection():
    global stop_flag
    stop_flag = False
    start_button.config(state=tk.DISABLED)
    stop_button.config(state=tk.NORMAL)
    _, frame = video_feed.read()
    hands, _ = detector.findHands(frame)

    if hands:
        if option_var.get() == 1:
            predicted_output = predict_numbers(hands[0])
        elif option_var.get() == 2:
            predicted_output = predict_alphabets(hands)
        else:
            predicted_output = predict_word(hands[0])
        label_var.set(f"Predicted: {predicted_output}")

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (640, 480))
    img = Image.fromarray(frame)
    imgtk = ImageTk.PhotoImage(image=img)
    label_img.imgtk = imgtk
    label_img.configure(image=imgtk)
    if not stop_flag:
        label_img.after(10, start_detection)

def stop_detection():
    global stop_flag
    stop_flag = True
    video_feed.release()
    start_button.config(state=tk.NORMAL)
    stop_button.config(state=tk.DISABLED)

# Load the saved models
number_model = joblib.load('ISL_Sign_Model.joblib')
alphabet_model = joblib.load('ISL_Alphabet_model.joblib')
word_model = joblib.load('ISL_Word_model.joblib')

# Dictionary to map predicted class to corresponding text for numbers
number_class_mapping = {
    1: "Number One",
    2: "Number Two",
    3: "Number Three",
    4: "Number Four",
    5: "Number Five",
    6: "Number Six",
    7: "Number Seven",
    8: "Number Eight",
    9: "Number Nine"
}

# Dictionary to map predicted class to corresponding text for alphabets
alphabet_class_mapping = {
   1: "This symbol is A",
    2: "This symbol is B",
    3: "This symbol is C",
    4: "This symbol is D",
    5: "This symbol is E",
    6: "This symbol is F",
    7: "This symbol is G",
    8: "This symbol is H",
    9: "This symbol is I",
    10: "This symbol is J",
    11: "This symbol is K",
    12: "This symbol is L",
    13: "This symbol is M",
    14: "This symbol is N",
    15: "This symbol is O",
    16: "This symbol is P",
    17: "This symbol is Q",
    18: "This symbol is R",
    19: "This symbol is S",
    20: "This symbol is T",
    21: "This symbol is U",
    22: "This symbol is V",
    23: "This symbol is W",
    24: "This symbol is X",
    25: "This symbol is Y",
    26: "This symbol is Z"
}

# Dictionary to map predicted class to corresponding text for words
word_class_mapping = {
    1: "Like",
    2: "Beautiful"
}

# Initializing video capture and hand detector
video_feed = cv2.VideoCapture(0)
detector = HandDetector(maxHands=2)  # Increase maxHands to 2 for detecting both hands

# Initialize Tkinter application
app = tk.Tk()
app.title("ISL Recognizer")

# Set background image
background_image = Image.open("C:/Users/mukhe/ISL Project/galaxy.jpg")  # Change the file path to your background image
background_image = background_image.resize((1369, 720), Image.BICUBIC)  # Resize the image as needed
background_photo = ImageTk.PhotoImage(background_image)
background_label = tk.Label(app, image=background_photo)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Create a frame for displaying the video feed
frame_container = tk.Frame(app, bd=5, relief=tk.RAISED, highlightbackground="Brown", highlightthickness=5)
frame_container.pack(padx=20, pady=8)

# Create a label for displaying the video feed
label_img = tk.Label(frame_container)
label_img.pack()

# Create a label for displaying the predicted sign or message
label_var = tk.StringVar()
label_var.set("Predicted: None")
label_prediction = tk.Label(app, textvariable=label_var, relief=tk.RAISED, font=('Helvetica', 16))
label_prediction.pack(pady=5)

# Create a variable to store the selected option
option_var = tk.IntVar()
option_var.set(0)

# Create a frame for option buttons
option_frame = tk.Frame(app)
option_frame.pack()

# Create radio buttons for selecting the option
option1_radio = tk.Radiobutton(option_frame, text="Numbers", variable=option_var, value=1, relief=tk.RAISED, borderwidth=3, bg="gray")
option1_radio.pack(side=tk.LEFT,padx=5, pady=5)
option2_radio = tk.Radiobutton(option_frame, text="Alphabets", variable=option_var, value=2, relief=tk.RAISED, borderwidth=3, bg="gray")
option2_radio.pack(side=tk.LEFT,padx=5, pady=5)
option3_radio = tk.Radiobutton(option_frame, text="Words", variable=option_var, value=3, relief=tk.RAISED,  borderwidth=3, bg="gray")
option3_radio.pack(side=tk.LEFT,padx=5, pady=5)

# Create a frame for start and stop buttons
button_frame = tk.Frame(app)
button_frame.pack()

# Create buttons to start and stop the sign detection process
start_button = tk.Button(button_frame, text="Start Detection", command=start_detection, bg="green", fg="white", relief=tk.RAISED, borderwidth=3)
stop_button = tk.Button(button_frame, text="Stop Detection", command=stop_detection, state=tk.DISABLED, bg="red", fg="white", relief=tk.RAISED, borderwidth=3)

# Pack buttons in a horizontal line
start_button.pack(side=tk.LEFT, padx=5, pady=5)
stop_button.pack(side=tk.LEFT, padx=5, pady=5)


# Variable to control the stop flag
stop_flag = False

# Start the Tkinter event loop
app.mainloop()
